In [16]:
import sys
from pathlib import Path
from datetime import datetime

# Añade src al path
sys.path.append(str(Path().resolve().parent / "src"))

import config as config

In [17]:
# # Configura el entorno de trabajo
%reload_ext autoreload
%autoreload 2

In [18]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2025-05-08 11:00:00')


In [ ]:
from data import load_raw_data, load_raw_data_v2

# Simulamos datos de producción muestreando datos históricos de hace 52 semanas (es decir, 1 año).
# Obtenemos los datos brutos de los últimos 28 días, para añadir redundancia a nuestra cadena de datos.
def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52) # 52 semanas anteriores
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}') 

    # Descargamos los 2 archivos del sitio web
    rides = load_raw_data_v2(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data_v2(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # Cambiamos los datos para fingir que son datos recientes
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [20]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2025-04-10 11:00:00'), to_date_=Timestamp('2024-05-09 11:00:00')
File 2024-04 was already in local storage
File 2024-05 was already in local storage


In [ ]:
from data import transform_to_time_series

# Transformar datos a time series
ts_data = transform_to_time_series(rides)

Rellenando huecos por localización: 100%|██████████| 254/254 [00:00<00:00, 1231.50it/s]


In [ ]:
# Convertir la columna pickup_hour a datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

In [ ]:
import hopsworks

# Conectar a Hopsworks
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# Conectar al Feature Store
feature_store = project.get_feature_store()

# Conectar al Feature Group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

2025-05-08 13:57:47,021 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 13:57:47,025 INFO: Initializing external client
2025-05-08 13:57:47,026 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 13:57:48,156 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1224869


In [ ]:
# Insertar datos en el Feature Group
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 170688/170688 | Elapsed Time: 00:10 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1224869/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions
2025-05-08 13:58:13,220 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-05-08 13:58:16,391 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-05-08 14:00:27,159 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-05-08 14:00:27,320 INFO: Waiting for log aggregation to finish.
2025-05-08 14:00:35,921 INFO: Execution finished successfully.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)